# Age and Gender Recognition
This Jupyter Notebook implements a machine learning model for age and gender recognition.

## Overview
The notebook includes the following steps:
- Data Preprocessing
- Model Training
- Evaluation and Results

## Dataset
Describe the dataset used (source, number of images, labels, etc.).


## Importing Required Libraries
We start by importing necessary libraries such as TensorFlow, OpenCV, and NumPy.

In [9]:
import cv2 as cv

## Data Preprocessing
Here, we load and preprocess the dataset for training. This includes resizing images, normalizing pixel values, and splitting the data into training and testing sets.

In [10]:
def faceBox(faceNet, frame):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]
    blob = cv.dnn.blobFromImage(frame, 1.0, (227, 227), [104, 117, 123], swapRB = False)
    faceNet.setInput(blob)
    detection = faceNet.forward()
    bbox = []
    for i in range(detection.shape[2]):
        confidence = detection[0,0,i,2]
        if confidence > 0.7:
            x1 = int(detection[0,0,i,3]*frameWidth)
            y1 = int(detection[0, 0, i, 4]*frameHeight)
            x2 = int(detection[0,0,i,5]*frameWidth)
            y2 = int(detection[0, 0, i, 6]*frameHeight)
            bbox.append([x1, y1, x2, y2])
            cv.rectangle(frame, (x1, y1), (x2,y2), (90,255,0), 1)
    return frame, bbox


## Model Architecture
The model architecture is defined using a deep learning framework such as TensorFlow/Keras. We use convolutional layers to extract features from images.

In [11]:
video = cv.VideoCapture(0)

## Training the Model
We compile and train the model using an appropriate loss function and optimizer.

In [12]:

faceProto = "opencv_face_detector.pbtxt"
faceModel = "opencv_face_detector_uint8.pb"

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

## Model Evaluation
We evaluate the model's performance on the test dataset using accuracy, precision, recall, and other metrics.

In [13]:
faceNet = cv.dnn.readNet(faceModel, faceProto)
ageNet = cv.dnn.readNet(ageModel, ageProto)
genderNet = cv.dnn.readNet(genderModel, genderProto)

In [14]:
MODEL_MEAN_VALUES = (78.42634, 87.7689, 114.89585)
ageList = ['(0-2)', '(2-4)', '(4-6)', '(6-10)', '(10-15)', '(15-18)', '(18-23)', '(23-26)', '(26-32)', '(32-37)', '(37-43)', '(43-50)', '(50-55)', '(55-60)', '(60+)']
genderList = ['Male', 'Female']

In [15]:
padding = 20

while True:
    ret, frame = video.read()
    frame, bbox = faceBox(faceNet, frame)
    for box in bbox:
        face = frame[max(0, box[1]-padding):min(box[3]+padding, frame.shape[0]-1), max(0, box[0]-padding):min(box[2]+padding, frame.shape[1]-1)]
        blob = cv.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
        
        genderNet.setInput(blob)
        gender_pred = genderNet.forward()
        gender = genderList[gender_pred[0].argmax()]

        ageNet.setInput(blob)
        age_pred = ageNet.forward()
        age = ageList[age_pred[0].argmax()]

        label = "{},{}".format(gender, age)
        cv.putText(frame, label, (box[0], box[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)


    cv.imshow("Age-Gender", frame)
    k = cv.waitKey(1)
    if k == ord('q'):
        break
video.release()
cv.destroyAllWindows()

## Results and Conclusion
We analyze the model's predictions and saw that the model was performing pretty good on gender classification. However, the model was giving correct age bracket of the individual, but in absence of adequate lightning, the age recognition was not working properly.